# Upload Structure

In [ ]:
from __future__ import print_function

from aiida import load_dbenv, is_dbenv_loaded
from aiida.backends import settings
if not is_dbenv_loaded():
    load_dbenv(profile=settings.AIIDADB_PROFILE)

from aiida.orm.querybuilder import QueryBuilder
from aiida.orm.data.structure import StructureData
from aiida.orm.calculation import Calculation

import ase.io
import numpy as np
import ipywidgets as ipw
from base64 import b64decode
from IPython.display import display, clear_output, Image
from fileupload import FileUploadWidget
from tempfile import NamedTemporaryFile

import nglview

## Step 1: Upload file

In [ ]:
def on_file_upload(c):
    global atoms
    with upload_out:
        clear_output()
        tmp = NamedTemporaryFile(suffix=file_upload.filename)
        f = open(tmp.name, "w")
        f.write(file_upload.data)
        f.close()
        traj = ase.io.read(tmp.name, index=":")
        tmp.close()
        #update_view()
        if len(traj) > 1:
            print("Error: Uploaded file contained more than one structure")

        atoms = traj[0]
        formula = atoms.get_chemical_formula()
        inp_descr.value = "upload(%s)" % formula

        # search for existing structures
        qb = QueryBuilder()
        qb.append(StructureData)
        qb.append(Calculation, filters={'extras.formula':formula}, descendant_of=StructureData)
        qb.order_by({Calculation:{'ctime':'desc'}})
        for n in qb.iterall():
            calc = n[0]
            print("Found existing calculation: PK=%d | %s"%(calc.pk, calc.get_extra("structure_description")))
            thumbnail = b64decode(calc.get_extra("thumbnail"))
            display(Image(data=thumbnail))
        refresh_structure_view()
    
#TODO: FileUploadWidget doesn't fire event when same file is uploaded twice
file_upload = FileUploadWidget("Upload Structure")
file_upload.observe(on_file_upload, names='data')
upload_out = ipw.Output()
viewer = nglview.NGLWidget()
clear_output()

display(file_upload, upload_out,ipw.VBox([viewer]))

## Step 2: Store in AiiDA Database

In [ ]:
def refresh_structure_view():
    global viewer, atoms
    if hasattr(viewer, "component_0"):
        #viewer.clear_representations()
        viewer.component_0.remove_ball_and_stick()
        viewer.component_0.remove_ball_and_stick()
        viewer.component_0.remove_ball_and_stick()
        viewer.component_0.remove_unitcell()
        cid = viewer.component_0.id
        viewer.remove_component(cid)

    viewer.add_component(nglview.ASEStructure(atoms)) # adds ball+stick
    viewer.add_unitcell()
    viewer.center()

In [ ]:
def on_click_store(b):
    global atoms
    with store_out:
        clear_output()
        #AiiDA requires structures to have cell
        if np.all(atoms.cell == 0.0):
            atoms.center(vacuum=0.1)
        s = StructureData(ase=atoms)
        # ensure that tags got correctly translated into kinds 
        for t1, k in zip(atoms.get_tags(), s.get_site_kindnames()):
            t2 = int(k[-1]) if k[-1].isnumeric() else 0
            assert t1==t2
        s.description = inp_descr.value
        s.store()
        print("Stored in AiiDA: "+repr(s))

inp_descr = ipw.Text(placeholder="Description (optional)")   
btn_store = ipw.Button(description='Store in AiiDA')
btn_store.on_click(on_click_store)
store_out = ipw.Output()
display(ipw.HBox([btn_store, inp_descr]), store_out)